# Update QLD Conservation and Sensitive Lists

### Sources
Queensland Nature Conservation Act 1992

**Conservation**
[Metadata - Qld Species (Open Data Portal)](https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife)
[Data](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv)

**Sensitive**
[Metadata - Queensland Confidential Species (Open Data Portal)](https://www.data.qld.gov.au/dataset/queensland-confidential-species)
[Data](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv)

**Codes**
[Metadata - Qld Species codes](https://www.data.qld.gov.au/dataset/conservation-status-of-queensland-wildlife/resource/6344ea93-cadf-4e0c-9ff4-12dfb18d5f14)
[Data](https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv)

In [94]:
# import essential libraries
import pandas as pd
# import numpy as np
import requests
import io
from ftfy import fix_encoding

# top level directory
projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/"
# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/"
#Species List and Species codes URLS
listurl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species.csv"
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"

 ## Conservation List

In [ ]:
print("Downloading Qld Species Status Codes")
response = requests.get(codesurl)
# Remove bad encoding
rtext = fix_encoding(response.text)
# speciescodes = pd.read_csv(rtext)
# speciescodes = pd.read_csv(io.StringIO(rtext))

In [ ]:
print("Downloading Qld Conservation List")
response = requests.get(listurl)
# Remove bad encoding
rtext = fix_encoding(response.text)
# conservationlist = pd.read_csv(io.StringIO(rtext))

Firstly, some minimal changes to some Qld Nature Conservation Act codes so that they are consistent with other states

In [95]:
# speciescodes = pd.read_csv(projectdir + "source-data/QLD/species-status-codes.csv",encoding='cp1252')
speciescodes = pd.read_csv(io.StringIO(rtext))
ncastatuscodes = speciescodes[speciescodes['Field'] == "NCA_status"][['Code','Code_description']]
ncastatuscodes['Code_description'] = ncastatuscodes['Code_description'].str.replace(" wildlife","")
ncastatuscodes.loc[ncastatuscodes['Code_description'] == "Critically endangered",'Code_description'] = "Critically Endangered"
ncastatuscodes.loc[ncastatuscodes['Code_description'] == "Near threatened",'Code_description'] = "Near Threatened"
ncastatuscodes


,Code,Code_description
17,C,Least concern
18,CR,Critically Endangered
19,E,Endangered
20,EX,Extinct
21,NT,Near Threatened
22,PE,Extinct in the wild
23,SL,Special least concern
24,V,Vulnerable
25,I,International


Read in the Conservation list and join to get the code descriptions. Change the field names to `sourceStatus` and `status` as required by the ALA's conservation list processing.

In [96]:
# conservationlist = pd.read_csv(projectdir + "source-data/QLD/species.csv",encoding='cp1252')
conservationlist = pd.read_csv(io.StringIO(rtext))
conservationlist = pd.merge(conservationlist,ncastatuscodes,left_on=['NCA_status'],right_on=['Code'],how="left")
conservationlist.drop(['Code'],axis=1,inplace=True)
conservationlist = conservationlist.rename(columns={'NCA_status':'sourceStatus','Code_description':'status'})
conservationlist

,Taxon_Id,Kingdom,Class,Family,Scientific_name,Common_name,Taxon_author,sourceStatus,EPBC_status,Significant,Confidential,Endemicity,status
0,706,animals,amphibians,Limnodynastidae,Adelotus brevis,tusked frog,"(Günther, 1863)",V,NaN,Y,N,QA,Vulnerable
1,690,animals,amphibians,Limnodynastidae,Lechriodus fletcheri,black soled frog,"(Boulenger, 1890)",C,NaN,N,N,QA,Least concern
2,677,animals,amphibians,Limnodynastidae,Limnodynastes convexiusculus,marbled frog,"(MacLeay, 1877)",C,NaN,N,N,QAI,Least concern
3,678,animals,amphibians,Limnodynastidae,Limnodynastes dumerilii,grey bellied pobblebonk,"Peters, 1863",C,NaN,N,N,QA,Least concern
4,679,animals,amphibians,Limnodynastidae,Limnodynastes fletcheri,barking frog,"Boulenger, 1888",C,NaN,N,N,QA,Least concern
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21798,25858,protozoans,slime molds,Physaraceae,Fuligo septica,NaN,(L.) F.H.Wigg.,C,NaN,N,N,Q,Least concern
21799,34854,protozoans,slime molds,Stemonitidaceae,Comatricha pulchella,NaN,(Bab.) Rostaf.,C,NaN,N,N,Q,Least concern
21800,27732,protozoans,slime molds,Stemonitidaceae,Stemonitis splendens,NaN,Rostaf.,C,NaN,N,N,Q,Least concern
21801,41204,protozoans,slime molds,Trichiidae,Hemitrichia serpula,NaN,(Scop.) Rostaf.,C,NaN,N,N,Q,Least concern


In [97]:
endemicitycodes = speciescodes[speciescodes['Field'] == "Endemicity"][['Code','Code_description']]
endemicitycodes

,Code,Code_description
0,IA,Naturalised from elsewhere in Australia
1,II,Naturalised from overseas
2,IU,Naturalised species - origin not defined
3,Q,Queensland endemic
4,QA,Australian endemic
5,QAI,Naturally occurs in Australia and overseas
6,QI,Naturally occurs in Queensland and overseas
7,U,Native to Queensland - endemicity not defined
8,VA,Australian vagrant
9,VI,International vagrant


In [98]:
# expand endemicity
endemicitycodes = speciescodes[speciescodes['Field'] == "Endemicity"][['Code','Code_description']]
conservationlist = pd.merge(conservationlist,endemicitycodes,left_on=['Endemicity'],right_on=['Code'],how="left")
conservationlist.drop(['Code'],axis=1,inplace=True)
conservationlist = conservationlist.rename(columns={'Code_description':'Endemicity_description'})
conservationlist

,Taxon_Id,Kingdom,Class,Family,Scientific_name,Common_name,Taxon_author,sourceStatus,EPBC_status,Significant,Confidential,Endemicity,status,Endemicity_description
0,706,animals,amphibians,Limnodynastidae,Adelotus brevis,tusked frog,"(Günther, 1863)",V,NaN,Y,N,QA,Vulnerable,Australian endemic
1,690,animals,amphibians,Limnodynastidae,Lechriodus fletcheri,black soled frog,"(Boulenger, 1890)",C,NaN,N,N,QA,Least concern,Australian endemic
2,677,animals,amphibians,Limnodynastidae,Limnodynastes convexiusculus,marbled frog,"(MacLeay, 1877)",C,NaN,N,N,QAI,Least concern,Naturally occurs in Australia and overseas
3,678,animals,amphibians,Limnodynastidae,Limnodynastes dumerilii,grey bellied pobblebonk,"Peters, 1863",C,NaN,N,N,QA,Least concern,Australian endemic
4,679,animals,amphibians,Limnodynastidae,Limnodynastes fletcheri,barking frog,"Boulenger, 1888",C,NaN,N,N,QA,Least concern,Australian endemic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21798,25858,protozoans,slime molds,Physaraceae,Fuligo septica,NaN,(L.) F.H.Wigg.,C,NaN,N,N,Q,Least concern,Queensland endemic
21799,34854,protozoans,slime molds,Stemonitidaceae,Comatricha pulchella,NaN,(Bab.) Rostaf.,C,NaN,N,N,Q,Least concern,Queensland endemic
21800,27732,protozoans,slime molds,Stemonitidaceae,Stemonitis splendens,NaN,Rostaf.,C,NaN,N,N,Q,Least concern,Queensland endemic
21801,41204,protozoans,slime molds,Trichiidae,Hemitrichia serpula,NaN,(Scop.) Rostaf.,C,NaN,N,N,Q,Least concern,Queensland endemic


In [99]:
#expand ebpc codes
epbccodes = speciescodes[speciescodes['Field'] == "EPBC_status"][['Code','Code_description']]
conservationlist = pd.merge(conservationlist,epbccodes,left_on=['EPBC_status'],right_on=['Code'],how="left")
conservationlist.drop(['Code'],axis=1,inplace=True)
conservationlist = conservationlist.rename(columns={'Code_description':'EPBC_status_description'})
conservationlist.drop(['EPBC_status'],axis=1,inplace=True)

**Species without a status**

In [100]:
# any without a status?
conservationlist[conservationlist['status'].isna()].groupby(["Class","Endemicity_description"]).size()

Class                 Endemicity_description                        
Agaricomycetes        Native to Queensland - endemicity not defined        3
                      Naturalised species - origin not defined             2
                      Queensland endemic                                   1
amphibians            Naturalised from overseas                            1
arachnids             Queensland endemic                                   1
birds                 International vagrant                                2
                      Naturalised from overseas                           14
bivalves              Native to Queensland - endemicity not defined        2
                      Naturally occurs in Australia and overseas           2
branchiopods          Australian endemic                                   1
                      Queensland endemic                                   1
cartilaginous fishes  Australian endemic                                  15
       

**Tidy up**
* rename fields to Darwin Core
* remove records with no status or Least Concern
* replace kingdom/class values with scientific names

In [101]:
conservationlist = conservationlist.rename(columns=
{
    'Taxon_Id':'taxonID',
    'Kingdom':'kingdom',
    'Class':'class',
    'Family':'family',
    'Scientific_name':'scientificName',
    'Common_name': 'vernacularName',
    'Taxon_author':'scientificNameAuthorship',
    'NCA_status':'sourceStatus',
    'EPBC_status_description':'epbcStatus',
    'Endemicity_description':'endemicity',
    'Significant':'significant',
    'Confidential':'confidential'
})
# remove empty or Least Concern status records
conservationlist = conservationlist[((conservationlist['status'] != "Least concern") & (conservationlist['status'].notna()))]

# Replace kingdom and class values with scientific terms
conservationlist.loc[conservationlist["kingdom"] == "animals", "kingdom"] = "Animalia"
conservationlist.loc[conservationlist["kingdom"] == "plants", "kingdom"] = "Plantae"
conservationlist.loc[conservationlist["class"] == "land plants", "class"] = "Equisetopsida"
conservationlist.loc[conservationlist["class"] == "amphibians", "class"] = "Amphibia"
conservationlist.loc[conservationlist["class"] == "birds", "class"] = "Aves"
conservationlist.loc[conservationlist["class"] == "cartilaginous fishes", "class"] = "Chondrichthyes"
conservationlist.loc[conservationlist["class"] == "insects", "class"] = "Insecta"
conservationlist.loc[conservationlist["class"] == "malacostracans", "class"] = "Malacostraca"
conservationlist.loc[conservationlist["class"] == "mammals", "class"] = "Mammalia"
conservationlist.loc[conservationlist["class"] == "ray-finned fishes", "class"] = "Actinopterygii"
conservationlist.loc[conservationlist["class"] == "reptiles", "class"] = "Reptilia"
conservationlist.loc[conservationlist["class"] == "snails", "class"] = "Gastropoda"
conservationlist.loc[conservationlist["class"] == "arachnids", "class"] = "Arachnida"
conservationlist.groupby(["kingdom","class"]).size()

kingdom   class         
Animalia  Actinopterygii       9
          Amphibia            41
          Aves               153
          Chondrichthyes       2
          Gastropoda           3
          Insecta              8
          Malacostraca        10
          Mammalia            67
          Reptilia            71
Plantae   Charophyceae         1
          Equisetopsida     1860
dtype: int64

In [102]:
#Write to CSV
print("Writing to CSV")
conservationlist.to_csv(projectdir + "current-lists/conservation-lists/QLD-conservation-2022-10.csv",encoding="UTF-8",index=False)
print("Processing finished")